# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Part 2: Dataset + Data Collection

## Overview

Based on the feedback you received from your lightning talk, choose **one** of your topic areas to move forward. For Part 2, you'll need to collect, clean, and document the dataset(s) you intend to use for your project.

This is not always a trivial task. Remember that data acquisition, transformation, and cleaning are typically the most time-consuming parts of data science projects, so don’t procrastinate!

Once you have your data, read into it and review it to confirm whether it is as productive as you intended. If not, switch datasets, gather additional data (e.g. multiple datasets), or revise your project goals.

Create your own database and data dictionary, then clean and munge your data as appropriate. Finally, document your work so far.

**Goal**: Find the data you need for your project, clean, and document it.


## Requirements

1. Find and Clean Your Data: Source and format the required data for your project.
   - Create a database
   - Create a data dictionary
2. Perform preliminary data munging and cleaning of your data: organize your data relevant to your project goals.
   - Review data to verify initial assumptions
   - Clean and munge data as necessary
3. Describe your data: keep your intended audience(s) in mind.
   - Document your work so far in a Jupyter notebook.

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver

import re
import time
import json

I am sourcing my data by webscraping pets4homes.co.uk. My main goal is to scrape information for each current listing for dogs and cats. Time permitting, I may also go back and scrape other animal listings too.

My plan to source the data requires two rounds a webscraping. The first round will loop through each page of search results and access the url associated with each listing. After removing any duplicates returned by this scrape, I will use the scraped urls to view each listing and scrape it's relevant information.

In [263]:
# importing a page of manually scraped html to be used for developing functions/code

path = '/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/dogs_html.rtf'

with open(path) as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

In [3]:
# function to extract urls from search pages

def get_url(page):
    '''    
    Extracts Pets4Homes advert listing URLs from a page of search results:
    
    Takes a BeautifulSoup object as input. Finds all instances of the class storing each an advert listing's
    URL. Adds each unique URL to a list. Returns the list.
    
    ---------
    Arguments
    ---------
    page : bs4.BeautifulSoup
        the html for a Pets4Homes search page, encoded as a BeautifulSoup object
    
    ---------
    Returns
    ---------
    cur_page_listing_urls : list
        a list containg the URLs for each advert listed in page
    '''
    
    cur_page_listing_urls = []

    # n.b. Pets4Home's HTML class names appear to change on a daily basis. 'ib iu' works as I am writing this
    # comment, but may not when you are reading it.
    
    for a in page.find_all('a', class_="ab Ym"):
        url = 'https://www.pets4homes.co.uk' + a['href']
        if url in cur_page_listing_urls:
            continue
        else:
            cur_page_listing_urls.append(url)
    
    return cur_page_listing_urls

In [ ]:
# testing get_url
get_url(soup)

### Scrape 1 - geting advert URLs

To do - refactor this section so scrape is a single loop, not one per animal type. 

#### Dogs

In [300]:
# getting dogs urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

for page_num in range(1,501):
    
    URL = f'https://www.pets4homes.co.uk/sale/puppies/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))    

# flattening list of lists and dropping duplicates
dogs = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
dogs = pd.DataFrame(dogs, columns = ['URLs'])

# exporting to csv
dogs.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/dogs_ulrs_10-01-23.csv')

In [301]:
# checking output of dog search scrape
dogs = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/dogs_ulrs_10-01-23.csv',
                   index_col='Unnamed: 0')
dogs

,URLs
0,https://www.pets4homes.co.uk/classifieds/bhhhe...
1,https://www.pets4homes.co.uk/classifieds/n-9id...
2,https://www.pets4homes.co.uk/classifieds/yblc-...
3,https://www.pets4homes.co.uk/classifieds/b4ei5...
4,https://www.pets4homes.co.uk/classifieds/j0vow...
...,...
7918,https://www.pets4homes.co.uk/classifieds/hjb6l...
7919,https://www.pets4homes.co.uk/classifieds/pqe3z...
7920,https://www.pets4homes.co.uk/classifieds/owqq6...
7921,https://www.pets4homes.co.uk/classifieds/pyigz...


#### Cats

In [390]:
# getting cats urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,219):
    
    URL = f'https://www.pets4homes.co.uk/sale/kittens/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {219-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
cats = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
cats = pd.DataFrame(cats, columns = ['URLs'])

# exporting to csv
cats.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/cat_ulrs.csv')

10 pages scraped, 209 to go. Time is: 11:45:51
20 pages scraped, 199 to go. Time is: 11:46:21
30 pages scraped, 189 to go. Time is: 11:47:00
40 pages scraped, 179 to go. Time is: 11:47:38
50 pages scraped, 169 to go. Time is: 11:48:28
60 pages scraped, 159 to go. Time is: 11:49:11
70 pages scraped, 149 to go. Time is: 11:49:47
80 pages scraped, 139 to go. Time is: 11:50:29
90 pages scraped, 129 to go. Time is: 11:51:02
100 pages scraped, 119 to go. Time is: 11:51:35
110 pages scraped, 109 to go. Time is: 11:52:09
120 pages scraped, 99 to go. Time is: 11:52:41
130 pages scraped, 89 to go. Time is: 11:53:19
140 pages scraped, 79 to go. Time is: 11:53:59
150 pages scraped, 69 to go. Time is: 11:54:44
160 pages scraped, 59 to go. Time is: 11:55:30
170 pages scraped, 49 to go. Time is: 11:56:16
180 pages scraped, 39 to go. Time is: 11:57:04
190 pages scraped, 29 to go. Time is: 11:58:02
200 pages scraped, 19 to go. Time is: 11:59:01
210 pages scraped, 9 to go. Time is: 11:59:55


#### Reptiles

In [433]:
# getting reptiles urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,112):
    
    URL = f'https://www.pets4homes.co.uk/sale/reptiles/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {112-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
reptiles = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
reptiles = pd.DataFrame(reptiles, columns = ['URLs'])

# exporting to csv
reptiles.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/reptiles_ulrs.csv')

10 pages scraped, 209 to go. Time is: 08:42:46
20 pages scraped, 199 to go. Time is: 08:43:06
30 pages scraped, 189 to go. Time is: 08:43:30
40 pages scraped, 179 to go. Time is: 08:44:01
50 pages scraped, 169 to go. Time is: 08:44:40
60 pages scraped, 159 to go. Time is: 08:45:29
70 pages scraped, 149 to go. Time is: 08:46:06
80 pages scraped, 139 to go. Time is: 08:46:43
90 pages scraped, 129 to go. Time is: 08:47:20
100 pages scraped, 119 to go. Time is: 08:48:05
110 pages scraped, 109 to go. Time is: 08:48:58


#### Birds

In [4]:
# getting birds urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,107):
    
    URL = f'https://www.pets4homes.co.uk/sale/birds/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {107-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
birds = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
birds = pd.DataFrame(birds, columns = ['URLs'])

birds.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/birds_ulrs.csv')

10 pages scraped, 97 to go. Time is: 11:21:11
20 pages scraped, 87 to go. Time is: 11:21:28
30 pages scraped, 77 to go. Time is: 11:21:45
40 pages scraped, 67 to go. Time is: 11:22:04
50 pages scraped, 57 to go. Time is: 11:22:25
60 pages scraped, 47 to go. Time is: 11:22:45
70 pages scraped, 37 to go. Time is: 11:23:05
80 pages scraped, 27 to go. Time is: 11:23:25
90 pages scraped, 17 to go. Time is: 11:23:43
100 pages scraped, 7 to go. Time is: 11:23:55


NameError: name 'reptiles' is not defined

#### Rodents

In [10]:
# getting rodents urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,86):
    
    URL = f'https://www.pets4homes.co.uk/sale/rodents/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {107-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
rodents = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
rodents = pd.DataFrame(rodents, columns = ['URLs'])

rodents.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/rodents_ulrs.csv')

10 pages scraped, 97 to go. Time is: 11:29:12
20 pages scraped, 87 to go. Time is: 11:29:24
30 pages scraped, 77 to go. Time is: 11:29:36
40 pages scraped, 67 to go. Time is: 11:29:51
50 pages scraped, 57 to go. Time is: 11:30:07
60 pages scraped, 47 to go. Time is: 11:30:24
70 pages scraped, 37 to go. Time is: 11:30:42
80 pages scraped, 27 to go. Time is: 11:30:59


URLs    https://www.pets4homes.co.uk/classifieds/juq3f...
Name: 0, dtype: object

#### Rabbits

n.b. although rabbits are rodents, they are listed separately on Pets4Homes.

In [13]:
# getting rabbits urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,86):
    
    URL = f'https://www.pets4homes.co.uk/sale/rabbits/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {85-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
rabbits = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
rabbits = pd.DataFrame(rabbits, columns = ['URLs'])

rabbits.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/rabbits_ulrs.csv')

10 pages scraped, 75 to go. Time is: 11:32:27
20 pages scraped, 65 to go. Time is: 11:32:41
30 pages scraped, 55 to go. Time is: 11:32:54
40 pages scraped, 45 to go. Time is: 11:33:11
50 pages scraped, 35 to go. Time is: 11:33:27
60 pages scraped, 25 to go. Time is: 11:33:44
70 pages scraped, 15 to go. Time is: 11:34:02
80 pages scraped, 5 to go. Time is: 11:34:13


#### Fish

In [16]:
# getting fish urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,35):
    
    URL = f'https://www.pets4homes.co.uk/sale/fish/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {34-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
fish = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
fish = pd.DataFrame(fish, columns = ['URLs'])

fish.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/fish_ulrs.csv')

10 pages scraped, 24 to go. Time is: 11:45:24
20 pages scraped, 14 to go. Time is: 11:45:36
30 pages scraped, 4 to go. Time is: 11:45:48


In [17]:
# getting poultry urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,25):
    
    URL = f'https://www.pets4homes.co.uk/sale/poultry/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {24-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
poultry = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
poultry = pd.DataFrame(poultry, columns = ['URLs'])

poultry.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/poultry_ulrs.csv')

10 pages scraped, 14 to go. Time is: 11:48:38
20 pages scraped, 4 to go. Time is: 11:48:50


In [18]:
# getting horses urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,15):
    
    URL = f'https://www.pets4homes.co.uk/sale/horses/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {14-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
horses = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
horses = pd.DataFrame(horses, columns = ['URLs'])

horses.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/horses_ulrs.csv')

10 pages scraped, 4 to go. Time is: 11:52:05


In [20]:
# getting invertebrates urls

# launching Chrome
dr = webdriver.Chrome()

listing_urls = []

n = 0

for page_num in range(1,10):
    
    URL = f'https://www.pets4homes.co.uk/sale/invertebrates/local/local/page-{page_num}/'
    
    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    listing_urls.append(get_url(page))
          
    n += 1
    if n % 10 == 0:
        print(f'{n} pages scraped, {9-n} to go. Time is:', time.ctime()[11:19])

# flattening list of lists and dropping duplicates
invertebrates = set([item for sublist in listing_urls for item in sublist])

# converting to DataFrame
invertebrates = pd.DataFrame(invertebrates, columns = ['URLs'])

invertebrates.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/invertebrates_ulrs.csv')

### Scrape 2 - getting listings' HTML

I had initially designed my 2nd scraper loop to pull up each advert listing one by one, extract the relevent data and add it to a DataFrame. This would allow me to only save the information I need, not the full html for each listing. 

Unfortunately, I discovered that Pets4Homes changes its HTML class names on a regular (I think daily) basis. This was causing problems because code that I had written to extract the relevant information on one day would stop working the next. Although this didn't happen, it is also possible that Pets4Homes could have updated their class names whilst I was in a middle of a scrape. Any scrapes after the change would return no/wrong information, but this wouldn't become clear until the scrape was finished (which would have been a considerable time waste).

Given the problems outlined above, I have changed my approach so that my 2nd scraper loop will save the full html for each advert. I will then process the saved html in a 3rd to extract relevant information. This will ensure that I can work on writing information extraction code without worrying about a Pets4Homes update making it obsolete.

The shortcoming of this approach is that the code written for the 3rd step will only work for the pages I've already scraped, not for future scrapes. It might be possible to write a 2nd scraper loop which is uneffected by Pets4Homes' changing html classes, but I felt that given the timeframe, the approach I have chosen was safer.

#### Dogs

In [360]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 7910

dogs_html = []

for page in dogs['URLs'][n:]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    dogs_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(n, 'Time is:', time.ctime())

13


In [361]:
dogs_html_strs = [str(i) for i in dogs_html]

dogs_html_strs_DF = pd.DataFrame(data = dogs_html_strs, columns = ['URLs'])
dogs_html_strs_DF.head()

In [413]:
len(cats)

3612

In [364]:
dogs_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/dogs_html_strs_(7910-to-7923)_11-01-23')

#### Cats

In [424]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 0

cats_html = []

for page in cats['URLs'][0:371]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    cats_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(n, 'Time is:', time.ctime()[11:19])
    
print(len(cats_html))

50 Time is: 20:46:04
100 Time is: 20:48:49
150 Time is: 20:51:00
200 Time is: 20:53:52
250 Time is: 20:57:16
300 Time is: 21:00:17
350 Time is: 21:03:47
371


In [425]:
cats_html_strs = [str(i) for i in cats_html]

cats_html_strs_DF = pd.DataFrame(data = cats_html_strs, columns = ['URLs'])
cats_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><style class=""vjs-styles..."
2,"<html lang=""en""><head><style class=""vjs-styles..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [426]:
cats_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/cats_listing_scrapes_(0-to-371).csv')

#### Reptiles

In [31]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 1584

reptiles = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/reptiles_ulrs.csv',
                      index_col = 'Unnamed: 0')

reptiles_html = []

for page in reptiles['URLs'][n:len(reptiles)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    reptiles_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(n, 'Time is:', time.ctime()[11:19])
    
print(len(reptiles_html))

1600 Time is: 13:12:26
1650 Time is: 13:13:38
1700 Time is: 13:14:46
1750 Time is: 13:15:55
1800 Time is: 13:17:17
1850 Time is: 13:18:57
1900 Time is: 13:20:55
1950 Time is: 13:23:11
2000 Time is: 13:25:51
2050 Time is: 13:29:05
2100 Time is: 13:31:43
2150 Time is: 13:32:53
612


In [32]:
reptiles_html_strs = [str(i) for i in reptiles_html]

reptiles_html_strs_DF = pd.DataFrame(data = reptiles_html_strs, columns = ['URLs'])
reptiles_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><link as=""script"" href=""..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [34]:
reptiles_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/reptiles_listing_scrapes_(1584-to-612).csv')

#### Birds

In [42]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 1465

birds = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/birds_ulrs.csv',
                      index_col = 'Unnamed: 0')

birds_html = []

for page in birds['URLs'][n:len(birds)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    birds_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(f'{n} scraped,{len(birds)-n} remaining. Time is: {time.ctime()[11:19]}')
    
print(len(birds_html))

1500 scraped,180 remaining. Time is: 15:29:53
1550 scraped,130 remaining. Time is: 15:31:21
1600 scraped,80 remaining. Time is: 15:32:58
1650 scraped,30 remaining. Time is: 15:34:57
215


In [44]:
len(birds_html)

215

In [43]:
len(birds) -

1680

In [45]:
birds_html_strs = [str(i) for i in birds_html]

birds_html_strs_DF = pd.DataFrame(data = birds_html_strs, columns = ['URLs'])
birds_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><link as=""script"" href=""..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><style class=""vjs-styles..."


In [46]:
birds_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/birds_listing_scrapes_(1465-to-end).csv')

#### Rodents

In [47]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 0

rodents = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/rodents_ulrs.csv',
                      index_col = 'Unnamed: 0')

rodents_html = []

for page in rodents['URLs'][n:len(rodents)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    rodents_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(f'{n} scraped,{len(rodents)-n} remaining. Time is: {time.ctime()[11:19]}')
    
print(len(rodents_html))

50 scraped,1624 remaining. Time is: 15:44:16
100 scraped,1574 remaining. Time is: 15:46:30
150 scraped,1524 remaining. Time is: 15:49:16
200 scraped,1474 remaining. Time is: 15:51:51
250 scraped,1424 remaining. Time is: 15:54:57
300 scraped,1374 remaining. Time is: 15:58:33
350 scraped,1324 remaining. Time is: 16:00:48
400 scraped,1274 remaining. Time is: 16:02:32
450 scraped,1224 remaining. Time is: 16:05:01
500 scraped,1174 remaining. Time is: 16:07:31
550 scraped,1124 remaining. Time is: 16:09:41
600 scraped,1074 remaining. Time is: 16:12:24
650 scraped,1024 remaining. Time is: 16:14:31
700 scraped,974 remaining. Time is: 16:16:18
750 scraped,924 remaining. Time is: 16:18:36
800 scraped,874 remaining. Time is: 16:21:28
850 scraped,824 remaining. Time is: 16:24:21
900 scraped,774 remaining. Time is: 16:26:01
950 scraped,724 remaining. Time is: 16:27:33
1000 scraped,674 remaining. Time is: 16:29:26
1050 scraped,624 remaining. Time is: 16:31:40
1100 scraped,574 remaining. Time is: 16:3

In [48]:
rodents_html_strs = [str(i) for i in rodents_html]

rodents_html_strs_DF = pd.DataFrame(data = rodents_html_strs, columns = ['URLs'])
rodents_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><link as=""script"" href=""..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [51]:
rodents_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/rodents_listing_scrapes_(all).csv')

#### Rabbits

In [75]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 1280

rabbits = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/rabbits_ulrs.csv',
                      index_col = 'Unnamed: 0')

rabbits_html = []

for page in rabbits['URLs'][n:len(rabbits)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    rabbits_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(f'{n} scraped,{len(rabbits)-n} remaining. Time is: {time.ctime()[11:19]}')
    
print(len(rabbits_html))

1300 scraped,69 remaining. Time is: 23:36:26
1350 scraped,19 remaining. Time is: 23:37:26
89


In [77]:
rabbits_html_strs = [str(i) for i in rabbits_html]

rabbits_html_strs_DF = pd.DataFrame(data = rabbits_html_strs, columns = ['URLs'])
rabbits_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><link as=""script"" href=""..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [76]:
len(rabbits_html) + 1280

1369

In [78]:
rabbits_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/rabbits_listing_scrapes_(1280-to-1369).csv')

#### Fish

In [84]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 332

fish = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/fish_ulrs.csv',
                      index_col = 'Unnamed: 0')

fish_html = []

for page in fish['URLs'][n:len(fish)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    fish_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(f'{n} scraped,{len(fish)-n} remaining. Time is: {time.ctime()[11:19]}')
    
print(len(fish_html))

350 scraped,311 remaining. Time is: 00:16:21
400 scraped,261 remaining. Time is: 00:17:18
450 scraped,211 remaining. Time is: 00:18:15
500 scraped,161 remaining. Time is: 00:19:09
550 scraped,111 remaining. Time is: 00:20:05
600 scraped,61 remaining. Time is: 00:20:58
650 scraped,11 remaining. Time is: 00:21:52
329


In [86]:
fish_html_strs = [str(i) for i in fish_html]

fish_html_strs_DF = pd.DataFrame(data = fish_html_strs, columns = ['URLs'])
fish_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><style class=""vjs-styles..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [85]:
len(fish_html) + 332

661

In [87]:
fish_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/fish_listing_scrapes_(332-to-661).csv')

#### poultry

In [89]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 0

poultry = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/poultry_ulrs.csv',
                      index_col = 'Unnamed: 0')

poultry_html = []

for page in poultry['URLs'][n:len(poultry)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    poultry_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(f'{n} scraped,{len(poultry)-n} remaining. Time is: {time.ctime()[11:19]}')
    
print(len(poultry_html))

50 scraped,411 remaining. Time is: 00:39:38
100 scraped,361 remaining. Time is: 00:40:32
150 scraped,311 remaining. Time is: 00:41:28
200 scraped,261 remaining. Time is: 00:42:22
250 scraped,211 remaining. Time is: 00:43:17
300 scraped,161 remaining. Time is: 00:44:22
350 scraped,111 remaining. Time is: 00:45:17
400 scraped,61 remaining. Time is: 00:46:19
450 scraped,11 remaining. Time is: 00:47:36
461


In [90]:
poultry_html_strs = [str(i) for i in poultry_html]

poultry_html_strs_DF = pd.DataFrame(data = poultry_html_strs, columns = ['URLs'])
poultry_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><link as=""script"" href=""..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [91]:
fish_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/poultry_listing_scrapes_(all).csv')

#### horses

In [92]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 0

horses = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/horses_ulrs.csv',
                      index_col = 'Unnamed: 0')

horses_html = []

for page in horses['URLs'][n:len(horses)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    horses_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(f'{n} scraped,{len(horses)-n} remaining. Time is: {time.ctime()[11:19]}')
    
print(len(horses_html))

50 scraped,219 remaining. Time is: 14:40:38
100 scraped,169 remaining. Time is: 14:42:10
150 scraped,119 remaining. Time is: 14:43:37
200 scraped,69 remaining. Time is: 14:45:11
250 scraped,19 remaining. Time is: 14:46:47
269


In [93]:
horses_html_strs = [str(i) for i in horses_html]

horses_html_strs_DF = pd.DataFrame(data = horses_html_strs, columns = ['URLs'])
horses_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><link as=""script"" href=""..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [94]:
horses_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/horses_listing_scrapes_(all).csv')

#### invertebrates

In [95]:
dr = webdriver.Chrome()

# I've had to carry out this scrape in multiple stages due to selenium/chrome crashing. 
# n is the number of URLs in the dogs list which I had scraped prior to the last crash.
n = 0

invertebrates = pd.read_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/scraped_urls/invertebrates_ulrs.csv',
                      index_col = 'Unnamed: 0')

invertebrates_html = []

for page in invertebrates['URLs'][n:len(invertebrates)]:
    
    URL = page

    # going to the URL
    dr.get(URL)

    # getting the html 
    html = dr.page_source

    page = BeautifulSoup(html, 'html.parser')
    
    invertebrates_html.append(page)
    
    # output to track progress
    n += 1 
    if n % 50 == 0:
        print(f'{n} scraped,{len(invertebrates)-n} remaining. Time is: {time.ctime()[11:19]}')
    
print(len(invertebrates_html))

50 scraped,118 remaining. Time is: 14:50:51
100 scraped,68 remaining. Time is: 14:52:10
150 scraped,18 remaining. Time is: 14:53:27
168


In [96]:
invertebrates_html_strs = [str(i) for i in invertebrates_html]

invertebrates_html_strs_DF = pd.DataFrame(data = invertebrates_html_strs, columns = ['URLs'])
invertebrates_html_strs_DF.head()

,URLs
0,"<html lang=""en""><head><link as=""script"" href=""..."
1,"<html lang=""en""><head><link as=""script"" href=""..."
2,"<html lang=""en""><head><link as=""script"" href=""..."
3,"<html lang=""en""><head><link as=""script"" href=""..."
4,"<html lang=""en""><head><link as=""script"" href=""..."


In [98]:
invertebrates_html_strs_DF.to_csv('/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/invertebrates_listing_scrapes_(all).csv')

## Data extraction

I've now scraped the full html all current listings on Pets4homes.co.uk. The next step its to process this html to extract relevant the data.

In [40]:
def extract_data(html):
    """
    TO DO
    """
    
    soup = BeautifulSoup(html, 'html.parser')
    
    data_dict = {}

    application = soup.find('script', attrs = {'type':"application/ld+json"}).text.replace('\\n','\n')
    application_json = json.loads(application, strict=False)
    
    # TITLE
    try:
        data_dict['title'] = soup.find('title').text.split('|')[0]
    except:
        pass
        
    # PRICE
    try:
        data_dict['price'] = (application_json['offers']['price'])
    except:
        pass
        
    # URL
    try:
        data_dict['url'] = (application_json['offers']['url'])
    except:
        pass
        
    # SELLER
    try:
        data_dict['seller_type'] = (application_json['offers']['seller']['type'])
    except:
        pass
    
    try:
        data_dict['seller_name'] = (application_json['offers']['seller']['name'])
    except:
        pass
        
        
    # VERIFICATION
    colour_dict = {'#69d4a1':1, '#c0ccda':0}

    try:
        for i in (soup.find(attrs = {'data-testid':"verification-status-field"})):
            els = i.find_all()
            colour = (els[0]['style'])
            data_dict[els[1].text] = colour_dict[colour[13:20]]
    except:
        pass

    # IMAGES
    try:
        data_dict['n_images'] = len(application_json['image'])
    except:
        pass
    
    # CATERGORY
    try:
        data_dict['category'] = (application_json['category']).replace(' ','').split('>')[1]
    except:
        pass
    
    
    # DETAILS 
    try:
        listing_details = soup.find(attrs = {"data-testid": 'listing-details'})
        listing_params = listing_details.find_all(attrs = {"data-testid": 'details-parameter'})

        for param in listing_params:

            param_text = re.sub("[\<].*?[\>]", "|", str(param))
            param_text = param_text.split("|")
            values = []
            for el in param_text:
                if el != '':
                    values.append(el)
            data_dict[values[0]] = values[1]
    except:
        pass

    # DESCRIPTION
    try:
        data_dict['description'] = soup.find('meta', attrs = {'name':"description"})['content']
    except:
        pass
    
    
    return(data_dict)

In [3]:
all_data = pd.DataFrame()

path = '/Users/lewis/Desktop/GA/DSI25-lessons/projects/project-capstone/data'

# BIRDS
all_data = all_data.append(pd.read_csv(f'{path}/data_birds_listing/birds_listing_scrapes_(0-to-1465).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_birds_listing/birds_listing_scrapes_(1465-to-end).csv',
                                      index_col = 'Unnamed: 0'))

print('Birds done.')



# CATS
all_data = all_data.append(pd.read_csv(f'{path}/data_cats_listings/cats_listing_scrapes_(0-to-371).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_cats_listings/cats_listing_scrapes_(370-to-3268).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_cats_listings/cats_listing_scrapes_(3268-to-3612).csv',
                                      index_col = 'Unnamed: 0'))

print('Cats done.')



# DOGS
all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(0-to-4368)_11-01-23',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(4368-to-5561)_11-01-23',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(5561-to-6984)_11-01-23',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(6984-to-7910)_11-01-23',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(7910-to-7923)_11-01-23',
                                      index_col = 'Unnamed: 0'))

print('Dogs done.')



# FISH
all_data = all_data.append(pd.read_csv(f'{path}/data_fish_listings/fish_listing_scrapes_(0-to-332).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_fish_listings/fish_listing_scrapes_(332-to-661).csv',
                                      index_col = 'Unnamed: 0'))

print('Fish done.')


# HORSES
all_data = all_data.append(pd.read_csv(f'{path}/horses_listing_scrapes_(all).csv',
                                      index_col = 'Unnamed: 0'))

print('Horses done.')


# INVERTEBRATES
all_data = all_data.append(pd.read_csv(f'{path}/invertebrates_listing_scrapes_(all).csv',
                                      index_col = 'Unnamed: 0'))


print('Invertebrates done.')


# POULTRY
all_data = all_data.append(pd.read_csv(f'{path}/poultry_listing_scrapes_(all).csv',
                                      index_col = 'Unnamed: 0'))


print('Poultry done.')


# POULTRY
all_data = all_data.append(pd.read_csv(f'{path}/poultry_listing_scrapes_(all).csv',
                                      index_col = 'Unnamed: 0'))

print('Poultry done.')


# RABBITS
all_data = all_data.append(pd.read_csv(f'{path}/data_rabbits_listings/rabbits_listing_scrapes_(0-to-537).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_rabbits_listings/rabbits_listing_scrapes_(537-to-923).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_rabbits_listings/rabbits_listing_scrapes_(923-to-1280).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_rabbits_listings/rabbits_listing_scrapes_(1280-to-1369).csv',
                                      index_col = 'Unnamed: 0'))


print('Rabbits done.')



# REPTILES
all_data = all_data.append(pd.read_csv(f'{path}/data_reptiles_listings/reptiles_listing_scrapes_(0-to-1584).csv',
                                      index_col = 'Unnamed: 0'))

all_data = all_data.append(pd.read_csv(f'{path}/data_reptiles_listings/reptiles_listing_scrapes_(1584-to-2196).csv',
                                      index_col = 'Unnamed: 0'))


print('Reptiles done.')



# RODENTS
all_data = all_data.append(pd.read_csv(f'{path}/rodents_listing_scrapes_(all).csv',
                                      index_col = 'Unnamed: 0'))


print('Rodents done.')


/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_birds_listing/birds_listing_scrapes_(0-to-1465).csv',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_birds_listing/birds_listing_scrapes_(1465-to-end).csv',


Birds done.


/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_cats_listings/cats_listing_scrapes_(0-to-371).csv',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_cats_listings/cats_listing_scrapes_(370-to-3268).csv',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_cats_listings/cats_listing_scrapes_(3268-to-3612).csv',


Cats done.


/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(0-to-4368)_11-01-23',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(4368-to-5561)_11-01-23',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_dogs_listings/dogs_html_strs_(5561-to-6984)_11-01-23',
/var/folders/z2/yfr_960x6

...

/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_rabbits_listings/rabbits_listing_scrapes_(0-to-537).csv',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_rabbits_listings/rabbits_listing_scrapes_(537-to-923).csv',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:97: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_rabbits_listings/rabbits_listing_scrapes_(923-to-1280).csv',
/var/fol

Rabbits done.


/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:109: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_reptiles_listings/reptiles_listing_scrapes_(0-to-1584).csv',
/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/data_reptiles_listings/reptiles_listing_scrapes_(1584-to-2196).csv',


Reptiles done.
Rodents done.


/var/folders/z2/yfr_960x69zdtf98dwdj7s6r0000gn/T/ipykernel_55226/2674055899.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(pd.read_csv(f'{path}/rodents_listing_scrapes_(all).csv',


In [41]:
pets_data = []

n = 0
el = 0

for pet in all_data['URLs']:
    el += 1
    if el % 50 == 0:
        print(f'{el} complete. {len(all_data["URLs"]) - el} to go.')
    try:
        pets_data.append(extract_data(pet))
    except:
        n += 1
        pass    
print(f'{n} failed. Done.')

50 complete. 20161 to go.
100 complete. 20111 to go.
150 complete. 20061 to go.
200 complete. 20011 to go.
250 complete. 19961 to go.
300 complete. 19911 to go.
350 complete. 19861 to go.
400 complete. 19811 to go.
450 complete. 19761 to go.
500 complete. 19711 to go.
550 complete. 19661 to go.
600 complete. 19611 to go.
650 complete. 19561 to go.
700 complete. 19511 to go.
750 complete. 19461 to go.
800 complete. 19411 to go.
850 complete. 19361 to go.
900 complete. 19311 to go.
950 complete. 19261 to go.
1000 complete. 19211 to go.
1050 complete. 19161 to go.
1100 complete. 19111 to go.
1150 complete. 19061 to go.
1200 complete. 19011 to go.
1250 complete. 18961 to go.
1300 complete. 18911 to go.
1350 complete. 18861 to go.
1400 complete. 18811 to go.
1450 complete. 18761 to go.
1500 complete. 18711 to go.
1550 complete. 18661 to go.
1600 complete. 18611 to go.
1650 complete. 18561 to go.
1700 complete. 18511 to go.
1750 complete. 18461 to go.
1800 complete. 18411 to go.
1850 complet

14750 complete. 5461 to go.
14800 complete. 5411 to go.
14850 complete. 5361 to go.
14900 complete. 5311 to go.
14950 complete. 5261 to go.
15000 complete. 5211 to go.
15050 complete. 5161 to go.
15100 complete. 5111 to go.
15150 complete. 5061 to go.
15200 complete. 5011 to go.
15250 complete. 4961 to go.
15300 complete. 4911 to go.
15350 complete. 4861 to go.
15400 complete. 4811 to go.
15450 complete. 4761 to go.
15500 complete. 4711 to go.
15550 complete. 4661 to go.
15600 complete. 4611 to go.
15650 complete. 4561 to go.
15700 complete. 4511 to go.
15750 complete. 4461 to go.
15800 complete. 4411 to go.
15850 complete. 4361 to go.
15900 complete. 4311 to go.
15950 complete. 4261 to go.
16000 complete. 4211 to go.
16050 complete. 4161 to go.
16100 complete. 4111 to go.
16150 complete. 4061 to go.
16200 complete. 4011 to go.
16250 complete. 3961 to go.
16300 complete. 3911 to go.
16350 complete. 3861 to go.
16400 complete. 3811 to go.
16450 complete. 3761 to go.
16500 complete. 3711

In [346]:
pets_df = pd.DataFrame.from_dict(pets_data)

## Cleaning data

In [347]:
# removing duplicate rows
print(f'{pets_df.shape[0] - pets_df.drop_duplicates().shape[0]} duplicate rows dropped') 
pets_df.drop_duplicates(inplace = True)

718 duplicate rows dropped


In [348]:
# checking columns
print(pets_df.columns)

Index(['title', 'price', 'url', 'seller_type', 'seller_name', 'Phone', 'Email',
       'Facebook', 'Google', 'n_images', 'category', 'Adv. ID',
       'Adv. Location', 'Advert Type', 'Advertiser', 'Breed', 'Pet Age: ',
       'Pet Colour', 'Sex', 'description', 'Health Checked', 'Is Microchipped',
       'Is Neutered', 'Is Vaccinated', 'Is Worm Treated', 'Pet Available',
       'Pets in litter', 'Registered', 'Is advertiser the orig. breeder',
       'Is KC Registered', 'Pet Viewable with Mother', 'Birth Year',
       'Category 1', 'Category 2', 'Gender', 'Height', 'Origin',
       'Level Jumping', 'Level Dressage'],
      dtype='object')


In [349]:
pets_df.columns = ['title', 
 'price', 
 'url', 
 'seller_type', 
 'seller_name', 
 'phone_verified', 
 'email_verified',
 'facebook_verified', 
 'google_verified', 
 'n_images', 
 'category',
 'advert_id', 
 'advert_location',
 'advert_type', 
 'advertiser_type', 
 'breed', 
 'pet_age', 
 'pet_colour', 
 'pet_sex',
 'description',
 'health_checked',
 'microchipped',
 'neutered',
 'vaccinated',
 'worm_treated',
 'pet_available',
 'pets_in_litter',
 'registered',
 'original_breeder',
 'kc_registered',
 'viewable_with_mother',
 'birth_year',
 'category_1',
 'category_2',
 'gender',
 'height',
 'origin',
 'level_jumping',
 'level_dressage']

print(pets_df.columns)

Index(['title', 'price', 'url', 'seller_type', 'seller_name', 'phone_verified',
       'email_verified', 'facebook_verified', 'google_verified', 'n_images',
       'category', 'advert_id', 'advert_location', 'advert_type',
       'advertiser_type', 'breed', 'pet_age', 'pet_colour', 'pet_sex',
       'description', 'health_checked', 'microchipped', 'neutered',
       'vaccinated', 'worm_treated', 'pet_available', 'pets_in_litter',
       'registered', 'original_breeder', 'kc_registered',
       'viewable_with_mother', 'birth_year', 'category_1', 'category_2',
       'gender', 'height', 'origin', 'level_jumping', 'level_dressage'],
      dtype='object')


In [350]:
pets_df.dtypes

title                    object
price                    object
url                      object
seller_type              object
seller_name              object
phone_verified          float64
email_verified          float64
facebook_verified       float64
google_verified         float64
n_images                  int64
category                 object
advert_id                object
advert_location          object
advert_type              object
advertiser_type          object
breed                    object
pet_age                  object
pet_colour               object
pet_sex                  object
description              object
health_checked           object
microchipped             object
neutered                 object
vaccinated               object
worm_treated             object
pet_available            object
pets_in_litter           object
registered               object
original_breeder         object
kc_registered            object
viewable_with_mother     object
birth_ye

In [351]:
# converting price dtype to float
pets_df.price = pets_df.price.astype(float)

In [352]:
# dropping advert type, as all values are the same
pets_df.drop(columns = 'advert_type', inplace = True)

In [353]:
# 9475 of the sellers have unique names
sum(pets_df['seller_name'].value_counts()==1)

# 3219 of the seller names appear more than once
sum(pets_df['seller_name'].value_counts()>1)

# of the sellers who's names appear more than once, 1153 are organizations
sum(pets_df[pets_df['seller_type'] == 'Organization']['seller_name'].value_counts()>1)

# I'm going to drop the seller_name column and replace it with a continuous numeric column indicating the 
# number of times that seller's name appeared in the dataset. 
seller_name_n_occurances = dict(pets_df['seller_name'].value_counts())
pets_df['seller_n_adverts'] = [seller_name_n_occurances[i] for i in pets_df['seller_name']]
pets_df.drop(columns = 'seller_name', inplace = True)

# Roughly 2/3rds of the sellers who's names appear more than once are individuals, it is likely that several 
# of these are multiple individuals with similar names (e.g. John S.). I will explore this relationship
# during EDA and will perform additional cleaning if it seems necessary.

In [354]:
# 6653 unique locations
pets_df['advert_location'].value_counts()

# converting all values to strings
pets_df['advert_location'] = pets_df['advert_location'].astype(str)

# taking only the last elemnt of each location
pets_df['advert_location'] = [i.split(',')[-1].replace(' ', '') for i in pets_df['advert_location']]

# 1168 unique locations
pets_df['advert_location'].value_counts()

London         1116
Birmingham      454
Manchester      386
Nottingham      251
Doncaster       218
               ... 
Corsham           1
Lifton            1
IsleofLewis       1
Keith             1
Bures             1
Name: advert_location, Length: 1168, dtype: int64

In [355]:
# the seller_type and advertiser_type columns contain similar information. Advertiser type has more details
# on the types of organisations, but includes some NaNs. I will combine these columns.

# no 'Organization' has advertiser_type == 'Individual', there are 68 NaNs.
pets_df[pets_df['seller_type'] == 'Organization']['advertiser_type'].value_counts(dropna = False)

# every 'Person' has advertiser_type == 'Individual', except 92 NaNs
pets_df[pets_df['seller_type'] == 'Person']['advertiser_type'].value_counts(dropna = False)

# Inferring any row with seller_type == 'Person' should have advertiser_type == 'Individual' & replacing NaNs
pets_df.loc[pets_df['seller_type'] == 'Person','advertiser_type'] = 'Individual'

# Looking at rows with seller_type == 'Organization' & advertiser_type == NaN
pets_df[(pets_df['seller_type'] == 'Organization') & (pets_df['advertiser_type'].isna())]

# These adverts all have similar/generic titles (e.g. "rabbits for sale"), NaNs in most columns
# and are not verified by any of the 4 verification methods. I suspect they're fake ads and so will drop them.
pets_df = pets_df[pets_df['advertiser_type'].notna()]

# advertiser_type now contains all information in seller_type (plus extra detail), dropping seller_type
pets_df.drop(columns = 'seller_type', inplace = True)

In [356]:
# 388 unique values for breed
pets_df.breed.value_counts()

# 92 NaNs
sum(pets_df.breed.isna())
pets_df[pets_df.breed.isna()]

# these adverts seem similar to the ones I removed in the previous cell. I.e. mostly NaNs, unverified, generic
# names, etc. Dropping them.
pets_df = pets_df[pets_df.breed.notna()]

In [357]:
# pet_age and birth_year contain similar information, but the latter is horse specific. I'll combine them.

# all rows with birth_year != NaN are horses
pets_df[pets_df.birth_year.notna()]['category'].value_counts()

# no horse rows have NaNs for birth_year
len(pets_df[pets_df['category'] == 'Horses'])

# Horses' age in years can be approximated from their birth_year
pets_df.loc[pets_df['category'] == 'Horses','pet_age'] = (2023 - pets_df[pets_df['category'] == 'Horses']['birth_year'].astype(int)).astype(str)+' years'

# birth_year can now be dropped
pets_df.drop(columns = 'birth_year', inplace = True)

# 34 rows remain with NaN for pet_age. I'll drop them.
pets_df['pet_age'].isna().sum()
pets_df = pets_df[pets_df['pet_age'].notna()]

In [358]:
import datetime

#dt = datetime.datetime.strptime(str_td, "%H:%M:%S")
pets_age_in_days = []
for row in [i.split(',') for i in pets_df['pet_age'].astype(str)]:
    num_days = 0
    for el in row:
        try:
            if el == 'Just Born Today':
                pass
            elif 'NaN' in el: # some rows have 'NaN years' / 'NaN months'
                num_days = np.nan
            elif 'year' in el:
                num_days += (int(re.findall('\d+', el)[0]) * 365)
            elif 'month' in el:
                # 30.436875 is the mean month length in the gregorian calendar
                num_days += (int(re.findall('\d+', el)[0]) * 30.436875)
            elif 'week' in el:
                num_days += (int(re.findall('\d+', el)[0]) * 7)
            elif 'day' in el:
                num_days += int(re.findall('\d+', el)[0])
        except Exception as e:
              print(el)
    pets_age_in_days.append(num_days)
    
pets_df['pets_age_in_days'] = pets_age_in_days

# after converting all ages to days, 3 missing values remain, which I'll drop
pets_df[pets_df['pets_age_in_days'].isna()]
pets_df = pets_df[pets_df['pets_age_in_days'].notna()]

In [359]:
# about half of the rows have NaNs for pet_available (the date from which the pet is available for collection)
pets_df['pet_available'].isna().sum()

# Given this and given that this is unlikely to be a good predictor of price, I'm going to drop the column.
pets_df = pets_df.drop(columns = 'pet_available')

In [366]:
# 'registered' is cat specific and indicates if the cat is regisitered with one of 3 ownership clubs
# e.g. The Governing Council of the Cat Fancy (no, seriously). 'kc_registered' indicates whether a dog is 
# registered with the UK Kennel Club. These two colums can be combined. Later I'll create interaction terms
# with the pet catergories.

# for dogs, we have yes and no values and no NaNs
pets_df[pets_df['category'] == 'Dogs']['kc_registered'].value_counts(dropna = False)

# for cats, we have 2344 NaNs and either 'TICA', 'GCCF', 'FIFe', or some combination of them
pets_df[pets_df['category'] == 'Cats']['registered'].value_counts(dropna = False)

# If a cat is registered with any club, I will encode this as 1
pets_df.loc[(pets_df['category'] == 'Cats') & (pets_df['registered'].notna()), 'registered'] = 1

# I will assume that cats with NaNs for registered are not registered with any club
pets_df.loc[(pets_df['category'] == 'Cats') & (pets_df['registered'].isna()), 'registered'] = 0

# for dogs, I'll move the kc_registered values over to registered
pets_df.loc[(pets_df['category'] == 'Dogs') & (pets_df['kc_registered'] == 'yes'), 'registered'] = 1
pets_df.loc[(pets_df['category'] == 'Dogs') & (pets_df['kc_registered'] == 'no'), 'registered'] = 0

# kc_registered can now be dropped
pets_df.drop(columns = 'kc_registered', inplace = True)

In [364]:
pets_df.loc[(pets_df['category'] == 'Dogs') & (pets_df['kc_registered'] == 'yes'), 'kc_registered']

5289     yes
5293     yes
5296     yes
5298     yes
5299     yes
        ... 
13192    yes
13195    yes
13197    yes
13199    yes
13207    yes
Name: kc_registered, Length: 2774, dtype: object

The following columns do not require any cleaning:
 - url (wont be used in analysis, kept for reference)
 - advert_id (wont be used in analysis, kept for reference)
 - n_images.

[6.0, 6.57, 7.14, 7.71, 8.29, 8.86, 9.43, 10.0]

#### Creating dummy variables

Several columns have yes/no values that can be converted into binary variables.

The columns 'pet_sex', 'gender' and 'neutered' will need more tailored processing. 'pet_sex' has three levels (male, female and mixed), 'gender' is specific to horse listings and also has three levels (mare, gelding and stallion). 'neutered' only has two levels, but a gelding is an neutered male horse. Futhermore, the horse rows all have NaNs for the 'neutered' column. I will sort these columns so that the information stored in 'gender' is in the 'pet_sex' and 'neutered' columns, and 'gender' can be dropped.

In [63]:
# if gender == gelding, neutered == yes, pet_sex == male
pets_df.loc[pets_df['gender'] == 'Gelding', 'neutered'] = 'yes'
pets_df.loc[pets_df['gender' ]== 'Gelding', 'pet_sex'] = 'Male'

# if gender == stallion, neutered == n, pet_sex == male
pets_df.loc[pets_df['gender'] == 'Stallion', 'neutered'] = 'no'
pets_df.loc[pets_df['gender' ]== 'Stallion', 'pet_sex'] = 'Male'

# spaying a mare is extremely rare as it a dangerous operation. Its typically only done in the UK as a 
# life-saving proceedure. As such, I will assume all listed mares have not been spayed.
pets_df.loc[pets_df['gender'] == 'Mare', 'neutered'] = 'no'
pets_df.loc[pets_df['gender' ]== 'Mare', 'pet_sex'] = 'Female'

pets_df.drop(columns = 'gender', inplace = True)

In [86]:
dummies = pd.get_dummies(pets_df[['category',
                                  'advertiser_type',
                                  'health_checked',
                                  'microchipped',
                                  'vaccinated',
                                  'worm_treated',
                                  'original_breeder',
                                  'viewable_with_mother',
                                  'pet_sex',
                                  'neutered']],
              drop_first = True,
              dummy_na = True)

dummies.mean()*100

health_checked_yes          48.929287
health_checked_nan          41.390643
microchipped_yes            39.069481
microchipped_nan            34.514456
vaccinated_yes              44.831305
vaccinated_nan              34.514456
worm_treated_yes            55.528167
worm_treated_nan            34.899605
original_breeder_yes        27.381503
original_breeder_nan        71.724952
viewable_with_mother_yes    35.844503
viewable_with_mother_nan    59.816156
pet_sex_Male                11.872850
pet_sex_Mixed               16.807888
pet_sex_nan                 62.887074
neutered_yes                 4.195553
neutered_nan                33.204950
dtype: float64

In [134]:
pets_df[pets_df['advert_type'].isna()]

,title,price,url,seller_type,seller_name,phone_verified,email_verified,facebook_verified,google_verified,n_images,...,original_breeder,kc_registered,viewable_with_mother,birth_year,category_1,category_2,height,origin,level_jumping,level_dressage
177,Ringnecks birds for sale,135.00,/classifieds/bxh6o20bc-baby-ringnecks-availabl...,Person,Al H.,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,Budgerigars birds for sale,15.00,/classifieds/8rnf67rdu-baby-budgies-bromyard/,Person,V L.,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,Lovebirds birds for sale,400.00,/classifieds/glv79zmg-lovebirds-bridgnorth/,Person,inesa l.,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,Cockatiels birds for sale,85.00,/classifieds/4wwe8cvit-cockatiels-manchester/,Person,jewan s.,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,Macaws birds for sale,2500.00,/classifieds/z0eeypsgu-hand-reared-baby-blue-g...,Organization,Exotic Feathers,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19045,Rat rodents for sale,20.00,/classifieds/ypbl95abx-rats-harwich/,Organization,Debbie G.,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19104,Guinea Pig rodents for sale,39.99,/classifieds/umsvaosng-baby-rosette-guinea-pig...,Organization,Emersons Pet Centre,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19798,Guinea Pig rodents for sale,20.00,/classifieds/t7qf4az41-pair-of-guinea-pigs-pur...,Organization,Newbarn Cavies,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20024,Pygmy Hedgehog rodents for sale,900.00,/classifieds/i-obdmmwo-10-month-old-male-afric...,Person,Shannon M.,NaN,NaN,NaN,NaN,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
pets_df.gender.isna()

Mare        130
Gelding      70
Stallion     55
Name: gender, dtype: int64

In [24]:
pets_df[pets_df['Phone'] == 0 ]['url'].iloc[0]

'/classifieds/3266687-indian-ringneck-blackburn/'

In [193]:
# getting a the html for a dog listing to build listing scraper
dr = webdriver.Chrome()

URL = dogs.iloc[0].values[0]

# going to the URL
dr.get(URL)

# getting the html 
html = dr.page_source

page = BeautifulSoup(html, 'html.parser')

print(page.prettify)

<bound method Tag.prettify of <html lang="en"><head><link as="script" href="https://launchpad.privacymanager.io/latest/launchpad.bundle.js" rel="preload"/><script async="" defer="" src="https://gdpr.privacymanager.io/latest/gdpr.bundle.js"></script><script async="" src="//c.amazon-adsystem.com/aax2/apstag.js"></script><script async="" crossorigin="anonymous" integrity="sha384-0UmWi2drUOFtiIl9tXA3B9fWn6Oz5K3vecq0zC8vAzesBQkPzPdlJKXnimDbxieW" src="https://www.gstatic.com/recaptcha/releases/5qcenVbrhOy8zihcc2aHOWD4/recaptcha__en.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/signals/config/855094041759271?v=2.9.91&amp;r=stable"></script><script async="" src="https://connect.facebook.net/signals/plugins/identity.js?v=2.9.91"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-ZBLMPSVFX8&amp;l=dataLayer&amp;cx=c" type="text/ja

#### Bonus

4. Document your project goals (revise from your initial pitch)
   - Articulate “Specific aim”
   - Outline proposed methods and models
   - Define risks & assumptions

5. Create a blog post of at least 500 words that describes your work so far. Link to it in your Jupyter notebook.


## Deliverable Format & Submission

- Table, file, or database with relevant text file or notebook description.

---

## Suggested Ways to Get Started

- Review your initial proposal topic and feedback, and revise accordingly.
- Spend time with your data and verify that it can help you accomplish the goals you set out to pursue.
- If not, document how you intend to either change those goals.
- Alternatively, go find some additional data and/or try another source.

---

## Useful Resources

- [Exploratory Data Analysis](http://insightdatascience.com/blog/eda-and-graphics-eli-bressert.html)
- [Best practices for data documentation](https://www.dataone.org/all-best-practices)

---

## Project Feedback + Evaluation

[Attached here is a complete rubric for this project.](./capstone-part-02-rubric.md)

Your instructors will score each of your technical requirements using the scale below:

Score  | Expectations
--- | ---
**0** | _Incomplete._
**1** | _Does not meet expectations._
**2** | _Meets expectations, good job!_

,URLs
0,https://www.pets4homes.co.uk/classifieds/bhhhe...
1,https://www.pets4homes.co.uk/classifieds/n-9id...
2,https://www.pets4homes.co.uk/classifieds/yblc-...
3,https://www.pets4homes.co.uk/classifieds/b4ei5...
4,https://www.pets4homes.co.uk/classifieds/j0vow...
...,...
7918,https://www.pets4homes.co.uk/classifieds/hjb6l...
7919,https://www.pets4homes.co.uk/classifieds/pqe3z...
7920,https://www.pets4homes.co.uk/classifieds/owqq6...
7921,https://www.pets4homes.co.uk/classifieds/pyigz...
